In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pymongo

In [39]:
# Establish the remote connection to the mongo data base:
# myclient = pymongo.MongoClient("mongodb+srv://axme100:{}@cluster0-5jopz.mongodb.net/test?retryWrites=true&w=majority".format(mongoPass))
myclient = pymongo.MongoClient()

# This is the name of the cluster stored on mongo atlas
mydb = myclient["finalProject"]

# Create a new colection called efe articles
mycol = mydb["article_prepared"]

In [40]:
# This functionality could be improved using generators, but first get model working
cursor = mycol.find()
X = []
y = []

for article in cursor:
    doc = article['bag_of_words']
    nivel = article['level_binary']
    X.append(doc[0])
    y.append(nivel)

In [41]:
# Break up the data into separate cateegories
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [42]:
# Instantiate the tfdif vectorizer
# The first parameter normalizes the text into unicode
# The second parameter allows the vectorizer to consider bigrams
tfidf = TfidfVectorizer(strip_accents='unicode', ngram_range=(1, 2))

In [43]:
# Vectorize the training data
tf_idf_train = tfidf.fit_transform(X_train)

In [44]:
# Vectorize the test data
tf_idf_test = tfidf.transform(X_test)

In [45]:
# Initiate a logistic regression model
model = LogisticRegression()

In [46]:
# Fit the model to the training data
model.fit(tf_idf_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
# Predict the test data
preds = model.predict(tf_idf_test)

In [48]:
# Calculate the accuracey of the predictions
accuracy = (preds==y_test).mean()

In [49]:
# Display the accuracey of the model
accuracy

0.8257372654155496

In [101]:
# Break up the data into separate cateegories
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [137]:
# This just gets every document in the database
def get_processed_corpus():
    cursor = mycol.find()
    for article in cursor:
        yield article['bag_of_words']

In [138]:
gen = get_processed_corpus()

In [131]:
gen2 = get_processed_levels()

In [139]:
# This just gets every document in the database
def get_processed_levels():
    cursor = mycol.find()
    for article in cursor:
        yield article['level_binary']

In [328]:
# This is to get the feature names
feature_names = tfidf.get_feature_names()
dense = corpus.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)